# Linear Regression on CPAC_N10_11_10_20 dataset

- **Model**: Linear Regression
- **Target(s)**: `TF_Pelvis_Moment_X_BWBH`
- **Features**: sag trunk angle
- **Results**: slope/intercept


## Libraries

In [2]:
# Standard library
import warnings
import os


# Third party
import numpy as np
import pandas as pd
from sklearn import linear_model

from IPython.display import display, Markdown


# Local
import utils

## Load Dataset

In [3]:
DATASET = "CPAC_N10_11_10_20"
DATASET_CSV = f"s3://cpac/ORIG/{DATASET}/CPAC10S_N10_11_10_20.csv"
DATASET_README = f"s3://cpac/ORIG/{DATASET}/READ_ME.xlsx"
RESULTS_DIR = f"results/{DATASET}"


df_orig = utils.load_dataset(DATASET_CSV)
df_orig.describe()

,M_Trial_Num,M_Mass,M_Mass_to_L5S1,M_sub_task_indices,M_sub_task_num,M_include_overall,M_Index,M_Sub,M_sub_task_num_overall,M_Index_overall,...,RWEO_01_02_00_00_INSOLE_RX_ML_threshF50_mm,RWEO_01_02_00_00_INSOLE_RY_AP_threshF50_mm,RWEF_03_00_00_00_INSOLE_LFORCE_threshF50_BW,RWEF_03_04_00_00_INSOLE_LX_ML_threshF50_BH,RWEF_03_04_00_00_INSOLE_LY_AP_threshF50_BH,RWEF_01_00_00_00_INSOLE_RFORCE_threshF50_BW,RWEF_01_02_00_00_INSOLE_RX_ML_threshF50_BH,RWEF_01_02_00_00_INSOLE_RY_AP_threshF50_BH,RWSF_SCALED_RINSOLE_BW,RWSF_SCALED_LINSOLE_BW
count,1.971017e+06,1.971017e+06,567.0,0.0,0.0,1.971017e+06,1.971017e+06,1.971017e+06,1971017.0,1.971017e+06,...,1.683991e+06,1.683991e+06,1.967609e+06,1.661377e+06,1.661377e+06,1.967609e+06,1.683991e+06,1.683991e+06,1.967609e+06,1.967609e+06
mean,6.942408e+01,1.053204e+01,0.0,NaN,NaN,8.154633e-01,2.716049e+03,5.396072e+00,0.0,9.236236e+04,...,4.899182e+01,1.319880e+02,4.390082e-01,2.785011e-02,6.597227e-02,4.570948e-01,2.698083e-02,7.262811e-02,4.262551e-01,3.744147e-01
std,2.367938e+01,5.845028e+00,0.0,NaN,NaN,3.879214e-01,2.566750e+03,2.934289e+00,0.0,6.388543e+04,...,9.183171e+00,5.335901e+01,3.258924e-01,5.077019e-03,3.046704e-02,3.370890e-01,5.108799e-03,2.956883e-02,3.576745e-01,3.189683e-01
min,1.000000e+00,0.000000e+00,0.0,NaN,NaN,0.000000e+00,1.000000e+00,1.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.407405e-02,-5.624677e-02
25%,4.900000e+01,5.000000e+00,0.0,NaN,NaN,1.000000e+00,5.980000e+02,3.000000e+00,0.0,3.568000e+04,...,4.514000e+01,8.828000e+01,1.586822e-01,2.579404e-02,3.957558e-02,1.723314e-01,2.458140e-02,4.846361e-02,1.368566e-01,1.042724e-01
50%,7.900000e+01,1.000000e+01,0.0,NaN,NaN,1.000000e+00,1.875000e+03,5.000000e+00,0.0,9.043000e+04,...,5.076000e+01,1.302100e+02,4.173078e-01,2.872237e-02,6.118229e-02,4.304055e-01,2.806366e-02,7.106925e-02,3.676134e-01,3.228916e-01
75%,8.700000e+01,1.500000e+01,0.0,NaN,NaN,1.000000e+00,4.215000e+03,8.000000e+00,0.0,1.451810e+05,...,5.473000e+01,1.777800e+02,6.735348e-01,3.100000e-02,9.114589e-02,6.958240e-01,3.011671e-02,9.697091e-02,6.673267e-01,6.007689e-01
max,9.600000e+01,2.300000e+01,0.0,NaN,NaN,1.000000e+00,1.411700e+04,1.000000e+01,0.0,2.368970e+05,...,7.911000e+01,2.677800e+02,2.068044e+00,4.425281e-02,1.506798e-01,2.041016e+00,4.326744e-02,1.504382e-01,1.981410e+00,1.950301e+00


## Clean-up dataset

- Remove samples based on `M_include_overall`

In [5]:
df = df_orig[df_orig["M_include_overall"] > 0]

# Weed out wonky subjects
#df = df[df["M_Sub"].isin([2,4,5,6,7,8,9])]
#RESULTS_DIR += "_nowonky"

print(f"Number of samples: {df.shape[0]:,d} (before clean-up: {df_orig.shape[0]:,d})")
print(f"Number of trials: {len(df['M_Trial_Name'].unique())} (before clean-up: {len(df_orig['M_Trial_Name'].unique())})")
print(f"Number of subjects: {len(df['M_Sub'].unique())}")

Number of samples: 1,607,292 (before clean-up: 1,971,017)
Number of trials: 174 (before clean-up: 174)
Number of subjects: 10


## Linear Regression

In [32]:
df_linreg = df[["TF_Pelvis_Moment_X_BWBH", "SWRF_05_12_00_00_TRUNK_ANGLE_VL_X_sag"]].dropna()
X = df_linreg["SWRF_05_12_00_00_TRUNK_ANGLE_VL_X_sag"].values.reshape(-1, 1)
y = df_linreg["TF_Pelvis_Moment_X_BWBH"].values


estimator = linear_model.LinearRegression()
estimator.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
estimator.coef_

array([0.00121295])

In [34]:
estimator.intercept_

-0.014837700062068568

In [38]:
estimator.predict(X[100:101]), y[100]

(array([-0.1027784]), -0.19516205451610896)

In [43]:
np.linalg.lstsq(np.hstack((X, np.ones_like(X))), y)

(array([ 0.00121295, -0.0148377 ]),
 array([883.82060204]),
 2,
 array([55680.12152307,   864.24368316]))

In [44]:
estimator.score(X, y)

0.7065912033269665